In [6]:
import pickle

import graphviz
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import dask.dataframe as dd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score

from py.consts import DATA_DIR

In [7]:
with open(DATA_DIR / "model.pkl", "rb") as f:
    clf_model = pickle.load(f)

In [8]:
data = pd.read_parquet(DATA_DIR / f"encoded_test.parquet")

print(f"{len(data)} rows")
data.head()
X = data.drop("y", axis=1)
y = data[["y"]]

X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.4)
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size=0.50)

print("y_train")
print(f"length: {len(y_train)}")
print(f"positive: {len(y_train[y_train.y == 1])} ({len(y_train[y_train.y == 1]) / len(y_train) * 100:.2f}%) negative: {len(y_train[y_train.y == 0])}")
print()
print("y_")
print(f"length: {len(y_)}")
print(f"positive: {len(y_[y_.y == 1])} ({len(y_[y_.y == 1]) / len(y_) * 100:.2f}%) negative: {len(y_[y_.y == 0])}")
print()
print("y_cv")
print(f"length: {len(y_cv)}")
print(f"positive: {len(y_cv[y_cv.y == 1])} ({len(y_cv[y_cv.y == 1]) / len(y_cv) * 100:.2f}%) negative: {len(y_cv[y_cv.y == 0])}")
print()
print("y_test")
print(f"length: {len(y_test)}")
print(f"positive: {len(y_test[y_test.y == 1])} ({len(y_test[y_test.y == 1]) / len(y_test) * 100:.2f}%) negative: {len(y_test[y_test.y == 0])}")

4385206 rows
y_train
length: 2631123
positive: 28195 (1.07%) negative: 2602928

y_
length: 1754083
positive: 18511 (1.06%) negative: 1735572

y_cv
length: 877041
positive: 9249 (1.05%) negative: 867792

y_test
length: 877042
positive: 9262 (1.06%) negative: 867780


In [9]:
y_predict = clf_model.predict(X_)
accuracy_score(y_, y_predict)

0.9894469075864711

In [10]:
class_names = list([str(v) for v in y.y.unique()])
feature_names = list(X.columns)

In [11]:
# cross validation
y_cv_predict = clf_model.predict(X_cv)
print(f"cross-validation dataset accuracy: {accuracy_score(y_cv, y_cv_predict)}")

# Generate a classification report
report = classification_report(y_, y_predict)
print("\nClassification Report:\n", report)

# Compute the confusion matrix
cm = confusion_matrix(y_, y_predict)
print("\nConfusion Matrix:\n", cm)

scores = cross_val_score(clf_model, X_train, y_train, cv=2)  # 5-fold cross-validation
print("\nCross-Validation Scores:", scores)

cross-validation dataset accuracy: 0.9894543128542451


/Users/simonverhoek/Documents/programming/projects/flitsers/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/simonverhoek/Documents/programming/projects/flitsers/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/simonverhoek/Documents/programming/projects/flitsers/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div


Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99   1735572
           1       0.00      0.00      0.00     18511

    accuracy                           0.99   1754083
   macro avg       0.49      0.50      0.50   1754083
weighted avg       0.98      0.99      0.98   1754083


Confusion Matrix:
 [[1735572       0]
 [  18511       0]]


/Users/simonverhoek/Documents/programming/projects/flitsers/venv/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/simonverhoek/Documents/programming/projects/flitsers/venv/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Cross-Validation Scores: [0.98928367 0.98928442]


In [ ]:

results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(index='param_max_depth', columns='param_min_samples_split', values='mean_test_score')
sns.heatmap(pivot_table, annot=True, fmt=".3f", cmap="YlGnBu")
plt.xlabel('min_samples_split')
plt.ylabel('max_depth')
plt.show()

In [12]:
plot_tree(clf_model, feature_names=feature_names, class_names=class_names)

InvalidParameterError: The 'decision_tree' parameter of plot_tree must be an instance of 'sklearn.tree._classes.DecisionTreeClassifier' or an instance of 'sklearn.tree._classes.DecisionTreeRegressor'. Got RandomForestClassifier(max_depth=5, min_samples_leaf=2) instead.

In [13]:
target = list([str(v) for v in y.y.unique()])
feature_names = list(X.columns)
dot_data = tree.export_graphviz(clf_model,
                                out_file=None,
                                feature_names=feature_names,
                                class_names=target,
                                filled=True, rounded=True,
                                special_characters=True)
graph = graphviz.Source(dot_data)

graph

AttributeError: 'RandomForestClassifier' object has no attribute 'tree_'

In [14]:
r = export_text(clf_model, feature_names=feature_names)
print(r)

InvalidParameterError: The 'decision_tree' parameter of export_text must be an instance of 'sklearn.tree._classes.DecisionTreeClassifier' or an instance of 'sklearn.tree._classes.DecisionTreeRegressor'. Got RandomForestClassifier(max_depth=5, min_samples_leaf=2) instead.